In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import iraklis7_nn.config as config

2026-02-02 15:08:45.880 | INFO     | iraklis7_nn.config:<module>:11 - PROJ_ROOT path is: /Users/iraklis/git/iraklis7_nn


In [3]:
input_path: Path = config.PROCESSED_DATA_DIR
output_path: Path = config.PROCESSED_DATA_DIR 
figures_path: Path = config.FIGURES_DIR
models_path: Path = config.MODELS_DIR

### Load data

In [4]:
ds_train_path = input_path.joinpath(config.TRAIN_SET)
ds_train = tf.data.Dataset.load(str(ds_train_path))
ds_val_path = input_path.joinpath(config.VAL_SET)
ds_val = tf.data.Dataset.load(str(ds_val_path))
ds_test_path = input_path.joinpath(config.TEST_SET)
ds_test = tf.data.Dataset.load(str(ds_test_path))

### Create model

In [5]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  #tf.keras.Input(shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu',),
  tf.keras.layers.Dense(10)
])

/Users/iraklis/git/iraklis7_nn/lib/python3.9/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [6]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 101,770 (397.54 KB)

 Trainable params: 101,770 (397.54 KB)

 Non-trainable params: 0 (0.00 B)

### Compile model

In [7]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)


### Fit model

In [8]:
model.fit(
    ds_train,
    epochs=12,
    validation_data=ds_val,
)

Epoch 1/12
438/438 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.6233 - sparse_categorical_accuracy: 0.8249 - val_loss: 0.1994 - val_sparse_categorical_accuracy: 0.9456
Epoch 2/12
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1762 - sparse_categorical_accuracy: 0.9508 - val_loss: 0.1458 - val_sparse_categorical_accuracy: 0.9590
Epoch 3/12
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.1221 - sparse_categorical_accuracy: 0.9651 - val_loss: 0.1217 - val_sparse_categorical_accuracy: 0.9651
Epoch 4/12
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0920 - sparse_categorical_accuracy: 0.9738 - val_loss: 0.1083 - val_sparse_categorical_accuracy: 0.9680
Epoch 5/12
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0730 - sparse_categorical_accuracy: 0.9798 - val_loss: 0.0998 - val_sparse_categorical_accuracy: 0.9701
Epoch 6/12
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0597 - sparse_categorical_accuracy: 0.9841 - val_loss: 0.0937 - val_sparse_categorical_accuracy: 0.9720
Epoc

### Save the model

In [9]:
model_path = models_path.joinpath(config.MODEL_NAME)
model.save(model_path)

### Make predictions

In [10]:
y_pred_full = model.predict(ds_test)

55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [11]:
model.get_metrics_result()

{'loss': 0.08103970438241959,
 'sparse_categorical_accuracy': 0.9764285683631897}

### Save predictions to file

In [12]:
pred_path  = output_path.joinpath(config.PRED_SET)
np.savetxt(str(pred_path), y_pred_full)